In [1]:
#!/usr/bin/env python

from datetime import datetime
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host':'cl-analytics.mwt2.org', 'port':9200}],
                   send_get_body_as = 'POST'
                  )

#
# Buckets : 
#
#  Split Table: Rows
#    Aggregation : Terms
#    Field       : computingsite
#    Order By    : Term
#    Order       : Ascending
#    Size        : 5
#
#  Split Rows:
#    Sub Aggregation : Terms
#    Field           : inputfileproject
#    Order By        : Term
#    Order           : Ascending
#    Size            : 100
#

triumf = ['ANALY_TRIUMF', 'TRIUMF', 'TRIUMF_HIMEM',
          'TRIUMF_MCORE', 'TRIUMF_MCORE_LOMEM']

query = {
  "size": 0,
  "query": {
    "constant_score": {
      "filter": {
        "bool": {
          "must": [
            {"range": {"endtime": {"from": "2016-11-01T00:00:00",
                                   "to":   "2016-12-01T00:00:00"}}},
            {"terms" : {"computingsite": triumf}},
            {"terms" : {"jobstatus" : ['finished', 'failed']}},
          ]
        }
      }
    }
  },
  "aggs": {
    "compsite": {  # bucket name
      "terms": {   
        "field": "computingsite",
        #"size": 5,
        "order": {
          "_term": "asc"
        }
      },
      "aggs": {
        "infileproj": {  # bucket name
          "terms": {
            "field": "inputfileproject",
            "size": 100,
            "order": {
              "_term": "asc"
            }
          },
          "aggs": {
            "cputime": {
              "sum": {
                "field": "cpuconsumptiontime"
                #"script": {
                #  "inline": "doc['cpuconsumptiontime'].value/3600",
                #  "lang": "expression"
                #}
              }
            },
            "walltime": {
              "sum": {
                "field": "wall_time"
                #"script": {
                #  "inline": "doc['wall_time'].value/3600",
                #  "lang": "expression"
                #}
              }
            },
            "stageIn": {
              "sum": {
                "field": "timeStageIn"
              }
            }
          }
        }
      }
    }
  }
}


# prepare a list of indexes
today = datetime.today()
this_year = today.year
this_month = today.month
#indices = ','.join('jobs_archive_{year}-{month}*'.format(year=this_year,month=month) for month in 

res = es.search(index="jobs_archive_2016-11*,jobs-archive_2016-12*", body=query, request_timeout=60)
for item in res['aggregations']['compsite']['buckets']:
    print '%s\t%s' % (item['key'], item['doc_count'])
    for site in item['infileproj']['buckets']:
        print '  %s : %s : %s : %s : %s' % (site['key'], site['doc_count'], 
                                            site['cputime']['value'], site['walltime']['value'], site['stageIn']['value'])


ANALY_TRIUMF	288918
  data10_hi:data10_hi : 173 : 862.0 : 113845.0 : 85354.0
  data11_hi : 25 : 0.0 : 2477.0 : 229.0
  data12_8TeV : 1143 : 2856453.0 : 3391101.0 : 173763.0
  data12_8TeV:data12_8TeV : 54 : 1100.0 : 36561.0 : 28717.0
  data13_hip : 191 : 4929.0 : 59686.0 : 28330.0
  data15_13TeV : 5507 : 11261086.0 : 15112290.0 : 1239820.0
  data15_13TeV:data15_13TeV : 13795 : 9953928.0 : 13986570.0 : 1012193.0
  data15_5TeV : 8707 : 1175851.0 : 3262263.0 : 722753.0
  data15_5TeV:data15_5TeV : 4 : 31.0 : 2354.0 : 1669.0
  data15_hi : 1683 : 534382.0 : 1511659.0 : 549187.0
  data15_hi:data15_hi : 4150 : 20113044.0 : 25815159.0 : 3122593.0
  data16_13TeV : 40559 : 92678599.0 : 118327366.0 : 10722189.0
  data16_13TeV:data16_13TeV : 11571 : 15551526.0 : 23164935.0 : 2681616.0
  data16_hip5TeV : 460 : 92670.0 : 665144.0 : 427801.0
  data16_hip8TeV : 1129 : 59240.0 : 455427.0 : 219285.0
  group : 351 : 77289.0 : 168164.0 : 32729.0
  mc12_14TeV:mc12_14TeV : 10 : 150.0 : 4790.0 : 3337.0
  mc12_